# MCP 사용하기 (.py 파일로 실행하기)

https://langchain-ai.lang.chat/langgraph/agents/mcp/   
https://innovationlab.fetch.ai/resources/docs/examples/mcp-integration/multi-server-agent-example

모델 컨텍스트 프로토콜(MCP)은 애플리케이션이 언어 모델에 도구와 컨텍스트를 제공하는 방식을 표준화하는 개방형 프로토콜입니다. LangGraph 에이전트는 `langchain-mcp-adapters`라이브러리를 통해 MCP 서버에 정의된 도구를 사용할 수 있습니다.

In [7]:
%pip install -qU langchain-mcp-adapters
%pip install -qU "mcp[cli]"

Note: you may need to restart the kernel to use updated packages.


In [11]:
# math_server.py
# ./mcp/math_server.py

In [12]:
# weather_server.py
# ./mcp/weather_server.py

# 먼저 실행이 필요
# python ./mcp/weather_server.py

## MCP 만들기

In [16]:
from mcp.server.fastmcp import FastMCP

mcp = FastMCP("Weather")

@mcp.tool()
@mcp.tool()
def get_weather(city: str) -> str:
    """Get the current weather for a city"""
    # This is a mock implementation
    return f"The weather in {city} is sunny and 25°C"

if __name__ == "__main__":
    mcp.run(transport="streamable-http")


Note: you may need to restart the kernel to use updated packages.


In [ ]:
from mcp.server.fastmcp import FastMCP

mcp = FastMCP("Math")

@mcp.tool()
def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b

@mcp.tool()
def multiply(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b

if __name__ == "__main__":
    mcp.run(transport="stdio")


## 메인

In [ ]:
import getpass
import os
import asyncio
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
from dotenv import load_dotenv

# 환경 설정
load_dotenv(override=True)

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

async def main():
    client = MultiServerMCPClient(
        {
            "math": {
                "command": "python",
                "args": ["./mcp/math_server.py"], 
                "transport": "stdio",
            },
            "weather": {
                "url": "http://localhost:8000/mcp",
                "transport": "streamable_http",
            }
        }
    )
    
    tools = await client.get_tools()
    print("*** TOOLS : ", tools, "\n\n")
    
    # 에이전트 만들기 
    agent = create_react_agent(
        "openai:gpt-4o-mini",
        tools
    )
    
    # math_response = await agent.ainvoke(
    #     {"messages": [{"role": "user", "content": "what's (3 + 5) x 12?"}]}
    # )

    # print(math_response, "\n\n")

    # weather_response = await agent.ainvoke(
    #     {"messages": [{"role": "user", "content": "what is the weather in nyc?"}]}
    # )

    # print(weather_response, "\n\n")

    while True:
        message = input("User: ")
        if message.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break
        response = await agent.ainvoke({"messages": [{"role": "user", "content": message}]})
        print("AI: "+response["messages"][-1].content, "\n\n")

# ------------------------------------------------------------------------------
if __name__ == "__main__":
    _set_env("TAVILY_API_KEY")
    _set_env("OPENAI_API_KEY")
    asyncio.run(main())
